
## 1. 선형 회귀모형의 이해

In [ ]:
# 데이터 불러오기
import pandas as pd
df_heights = pd.read_csv('data/heights.csv')
df_heights

In [ ]:
# 또 다른 예제 데이터 
df_ins = pd.read_csv('data/insurance.csv')
df_ins_test = pd.read_csv('data/insurance_test.csv')
df_ins.head()

In [ ]:
df_ins_test

<br>

### 1.1. 단순 선형 회귀 모형의 적합 

**sklearn** 라이브러리의 linear_model 모듈에 있는 `LinearRegression()`로 선형 회귀모형을 적합

In [ ]:
# 함수 불러오기
from sklearn.linear_model import LinearRegression

In [ ]:
# 아들키를 아빠키로 설명하는 선형 회귀 모형 적합
model = LinearRegression()
model.fit(df_heights[['father']], df_heights['son']) # model.fit(X(입력변수), y(출력변수))

In [ ]:
# 적합된 회귀계수 확인
print(model.intercept_)
print(model.coef_)


**아들키-아빠키 관계식**: 
$ son = 86.072 + 0.514 \times father $

<br>


### [참고] `statsmodel` 라이브러리를 활용한 선형 회귀 분석

In [ ]:
# statsmodel 라이브러리 불러오기
import statsmodels.api as sm

In [ ]:
# 출력변수와 입력변수를 각각 y와 x로 할당하기
y = df_heights.son
x = df_heights.father

In [ ]:
# 입력변수 x에 상수항 추가하기
x = sm.add_constant(x)

In [ ]:
# Ordinary least square 모델을 이용해서 선형 회귀 분석 
model2 = sm.OLS(y, x).fit()
print(model2.summary())

#### [참고] 회귀 직선의 시각화 

In [ ]:

import seaborn as sns
mean_f = df_heights['father'].mean()
mean_s = df_heights['son'].mean()

plot = sns.scatterplot(data=df_heights,
                       x='father',
                       y='son')
plot.axhline(mean_s)
plot.axvline(mean_f)

plot.axline((mean_f, mean_s), slope=model.coef_[0], color='red') #model.coef_는 np.array이므로 사용할 객체를 선택해야 함. 

<br>
<br>

### 1.2. 다중 선형 회귀 모형 적합 


In [ ]:
# 수치형 설명 변수를 활용한 선형 회귀 모형 적합
linear_model = LinearRegression()
linear_model.fit(df_ins[['age','bmi','children']], df_ins['charges'])

In [ ]:
# 일반적인 명령어 구성
    #  X : 설명변수/독립변수,  y : 관심변수/종속변수

X = df_ins[['age','bmi','children']]
y = df_ins['charges']
    
    # 모형 설정
linear_model = LinearRegression()

    # 모형 적합
linear_model.fit(X, y)

In [ ]:
# 적합된 회귀계수 확인
print(linear_model.intercept_)
print(linear_model.coef_)

$ charges = -6916.24 + 239.99 \times age + 332.08\times bmi + 542.86\times children $

<br>

### 1.3. 범주형 변수의 가변수화

In [ ]:
df_ins

In [ ]:
# pandas의 get_dummies() 활용
df_dummies = pd.get_dummies(data=df_ins, drop_first=True)
df_dummies
  ## 범주형 변수의 수치화 -> 회귀모형 적합 가능

In [ ]:
#  회귀모형 재적합
linear_model = LinearRegression()
    
X = df_dummies.drop(columns=['charges'])
y = df_dummies['charges']

linear_model.fit(X, y)

print(linear_model.intercept_)
print(linear_model.coef_)

In [ ]:
# 회귀 계수 정리
coeff = pd.DataFrame(linear_model.coef_, X.columns, columns=['Coefficient'])
coeff

회귀 모형식

$ charges = -11938.54 \\ 
\qquad\qquad + 256.86\times age \\
\qquad\qquad + 339.19\times bmi \\
\qquad\qquad + 475.50\times children \\
\qquad\qquad - 131.31\times sex\_male \\
\qquad\qquad + 23848.53\times smoker\_yes\\
\qquad\qquad - 352.96\times region\_northwest	\\
\qquad\qquad - 1035.02\times region\_southeast	\\
\qquad\qquad - 960.05\times region\_southwest$

In [ ]:
# 단순 집계와 모형의 비교 - 변수 sex
df_ins.groupby('sex')['charges'].mean()


In [ ]:
sns.boxplot(x='sex', 
            y='charges', 
            data=df_ins)

단순 집계와 회귀 모형의 결과가 다름
* 집계 : 남자 > 여자
* 회귀 모형 : 남자 < 여자

In [ ]:
## 성별 흡연률의 확인
agg1_i = pd.crosstab(df_ins['sex'], df_ins['smoker'], normalize='index')
sns.heatmap(agg1_i, cmap='Blues', annot=True, fmt='.2f')

### 1.4. 선형 회귀 모형의 한계

선형 회귀 모형은 비선형의 관계를 반영 어려움

In [ ]:
# 회귀 모형의 한계 - 변수 bmi와 charges의 산점도
sns.scatterplot(data=df_ins, 
                x='bmi', 
                y='charges', 
                hue='smoker',
                alpha=0.5)

<br>

### 1.5. 변수 선택의 개념

각 회귀 계수에 대한 검정 결과를 활용해서 모형에 필요한 변수를 선택할 수 있습니다. 

In [ ]:
# statsmodels를 활용한 회귀모형 적합과 변수 선택
    # 상수항 추가
import statsmodels.api as sm
X_Sm = sm.add_constant(X)
X_Sm

In [ ]:
# 선형회귀 모형 적합 및 각 변수에 대한 검정
ls=sm.OLS(y,X_Sm).fit()
ls.summary()


<br>

### 1.6. 모형 활용 예측

적합된 모형을 활용하여 실제값이 없는 신규 데이터에 대해 `predict()`를 활용해 예측값을 계산할 수 있습니다. 

In [ ]:
# 신규 데이터 5건 
df_ins_test

In [ ]:
# 동일한 처리
df_dummies_test = pd.get_dummies(data=df_ins_test, drop_first=True)
df_dummies_test

In [ ]:
# 예측 값 계산
linear_model.predict(df_dummies_test)

In [ ]:
# 예측값 변수 추가
df_ins_test['pred'] = linear_model.predict(df_dummies_test)
df_ins_test

<br>

### 1.7. 결정계수 확인 

모형의 성능을 평가하기위해 평가지표를 설정하고 값을 계산할 수 있습니다.

In [ ]:
from sklearn.metrics import r2_score
y_pred = linear_model.predict(X)
r2_score(y, y_pred)

<br>
<br>

## 2. 로지스틱 회귀모형

<br>

### 2.1. 예제 데이터 불러오기, 전처리

In [ ]:
# 예제 데이터 불러오기
df_admit = pd.read_csv('data/students.csv', dtype={'admit':'category', 'rank':'category'})
df_admit

In [ ]:
# 테스트 데이터 불러오기
df_admit_new = pd.read_csv('data/students_new.csv', dtype={'rank':'category'})
df_admit_new

In [ ]:
# 설명변수, 관심변수 분할
X = pd.get_dummies(data=df_admit.drop(columns=['admit']), drop_first=True)
y = df_admit['admit']

In [ ]:
X

<br>

### 2.2. 로지스틱 회귀모형 적합

In [ ]:
from sklearn.linear_model import LogisticRegression
model_logit = LogisticRegression(max_iter=1000)
model_logit.fit(X, y)

In [ ]:
# 회귀계수 확인
model_logit.coef_

<br>

### 2.3. 새로운 관측치에 대한 예측


In [ ]:
# 테스트 데이터에 대한 동일한 처리 
X_new = pd.get_dummies(data=df_admit_new, drop_first=True)
X_new

In [ ]:
# 예측값 생성
model_logit.predict_proba(X_new)

In [ ]:
# 1일 확률만 선택
model_logit.predict_proba(X_new)[:, 1]

In [ ]:
# 기존 데이터에 예측 확률 변수 추가
df_admit_new['prob'] = model_logit.predict_proba(X_new)[:, 1]
df_admit_new


<br>
<hr>
<br>

## 3. 의사결정나무

관심 변수가 있는 일반적인 정형 데이터에 활용할 수 있는 대표적인 지도학습 알고리즘


<br>

### 3.1. 회귀 나무(Regression Tree) : 관심변수가 수치형인 경우

In [ ]:
# 데이터 불러오기
df_ins = pd.read_csv('data/insurance.csv')
df_ins_test = pd.read_csv('data/insurance_test.csv')

In [ ]:
# 데이터 살펴보기
df_ins.head()

In [ ]:
# 예측 대상 데이터 살펴보기
df_ins_test.head()

<br>

#### 3.1.1. 데이터 처리 

가변수를 생성하고, X, y로 데이터를 분할

In [ ]:
# 데이터 처리
    # X : 타겟 변수를 제외하고 범주형 변수를 가변수(dummy variable)로 변환
    # y : 타겟 변수만 선택
X = pd.get_dummies(data=df_ins.drop(columns=['charges']), drop_first=True)
y = df_ins['charges']

In [ ]:
X.head()

In [ ]:
y.head()

<br>

#### 3.1.2. 의사결정나무 모형의 적합 

**sklearn** 라이브러리의 tree 모듈에서 `DecisionTreeRegressor()`를 가져와서 의사결정 나무 모형을 적합하고, `plot_tree()`로 적합된 모형을 시각화


In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

model_tree = DecisionTreeRegressor(max_depth=4)
model_tree.fit(X, y)

In [ ]:
type(list(X.columns))

In [ ]:
# 모형 적합 결과의 시각화
import matplotlib.pyplot as plt
plt.figure(figsize=(100,12)) # 이미지 크기 조정

plot_tree(model_tree, 
          feature_names=list(X.columns),  
          filled=True, 
          fontsize=20)
plt.show()

<br>

#### 3.1.3. 예측값 계산

`predict()`로 예측값 계산 가능

In [ ]:
# 예측값 계산
df_dummies_test = pd.get_dummies(data=df_ins_test, drop_first=True)
model_tree.predict(df_dummies_test)

<br>

#### 3.1.4. 변수 중요도 확인
각 분기 과정에서의 변수별 설명량을 바탕으로 변수 중요도를 계산할 수 있습니다. 

In [ ]:
model_tree.feature_importances_

In [ ]:
sns.barplot(y=model_tree.feature_names_in_,
            x=model_tree.feature_importances_)

<br>
<br>
<br>

### 3.2. 판별 나무( Tree) : 관심변수가 범주형인 경우

In [ ]:
# 예제 데이터 불러오기
df_admit = pd.read_csv('data/students.csv', dtype={'admit':'category', 'rank':'category'})
df_admit

<br>

#### 3.2.1. 데이터 처리

In [ ]:
# 설명변수, 관심변수 분할
X = pd.get_dummies(data=df_admit.drop(columns=['admit']), drop_first=True)
y = df_admit['admit']

<br>

#### 3.2.2. 판별 모형 적합

In [ ]:
# 판별 모형 적합
from sklearn.tree import DecisionTreeClassifier

model_tree_c = DecisionTreeClassifier(max_depth=3)
model_tree_c.fit(X, y)

In [ ]:
# 모형 적합 결과의 시각화
plt.figure(figsize=(27,12))
plot_tree(model_tree_c, 
          feature_names=list(X.columns),  
          filled=True, 
          fontsize=15,
          impurity=False, 
          proportion=True)
plt.show()

#### End of script